In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from torch.autograd import Variable
from torch.autograd.function import Function
from torchsummary import summary
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator

In [48]:
w1 = torch.randn(8, 8, dtype=torch.double, requires_grad=True)
x = torch.randn(1, 8, dtype=torch.double)
target_data = torch.randn(1, 8, dtype=torch.double)

In [49]:
class OwnLossFunction(Function):
    @staticmethod
    # bias is an optional argument
    def forward(ctx, output, target_data):
        # Select and transform data
        ctx.save_for_backward(output, target_data)
        
        # Compute loss
        term = (target_data-output)**2
        loss_batches = torch.sum(input=term, dim=1)
        return loss_batches
    
    @staticmethod
    def backward(ctx, grad_input):
        """
        grad_output = 1, because it is initialize by Optimizer 
        In the backward pass we need to compute the gradient of the loss
        with respect to the input.
        """
        output, target_data = ctx.saved_tensors # this is a tensor of models output
        grad_output = 2.*(output-target_data)
        #print(grad_output)
        return grad_output, None

In [50]:
def loss_function(target_data, output):
    # Compute loss
    term = (target_data-output)**2
    loss_batches = torch.sum(input=term, dim=1)
    return loss_batches

In [51]:
criterion = nn.MSELoss()

In [52]:
for t in range(500):
    myloss = OwnLossFunction.apply 
    y_pred = F.relu(x.mm(w1))
    #loss = myloss(y_pred, target_data)
    #loss = criterion(y_pred, target_data)
    loss = loss_function(y_pred, target_data)
    print(t, loss.item())
    loss.backward()
    with torch.no_grad():
        w1 += 0.1 * w1.grad
        w1.grad.zero_()

0 90.40541438298683
1 457.6672597259482
2 2617.4367057301947
3 15334.150242345073
4 90210.10678090519
5 531079.4281996798
6 3126916.163429298
7 18411192.089643948
8 108404945.324668
9 638287790.9644607
10 3758235787.466637
11 22128476645.37703
12 130292378593.73589
13 767160984464.8623
14 4517040693296.744
15 26596316859650.14
16 156599003314347.03
17 922054281743863.8
18 5429051785059226.0
19 3.196623438384817e+16
20 1.882170554156855e+17
21 1.1082212413249108e+18
22 6.525202070616257e+18
23 3.842036271698884e+19
24 2.2621893626132208e+20
25 1.3319761580641026e+21
26 7.842670091957931e+21
27 4.617760896012322e+22
28 2.7189356995401853e+23
29 1.600908211730491e+24
30 9.426140908074973e+24
31 5.550107855517859e+25
32 3.2679012024414835e+26
33 1.9241388720583807e+27
34 1.132932169491557e+28
35 6.670699913129254e+28
36 3.9277053409996085e+29
37 2.312631275071111e+30
38 1.3616763351896167e+31
39 8.017544611639018e+31
40 4.7207269406404545e+32
41 2.7795620638938774e+33
42 1.6366049899064533